# Copyright (c) 2026 [Vital de Nodrest]

The code (Python, shell and PowerShell cells) is under the MIT license. Feel free to share and experiment! See LICENSE-CODE.txt in the project root for more information.
Due to their time-consuming and didactic nature, text & image contents are under the CC BY-NC-SA 4.0 license. Removal of the author's name or redistribution without credit is prohibited. See LICENSE-DOCS.txt in the project root for more information.

# Introduction to PINNs for Helmholtz problems

This notebook aims at being a basic introduction to PINNs using the example of Helmholtz wave propagation problems.

## PINNs

### Universal approximation theorem and PDE problems

### Automatic differentiation and physical losses

## Configuration

You can use any recent Python environment to run this notebook (ipykernel will be required for interactive computing).

The following cells are necessary regardless of your device.

In [5]:
pip install matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 10.8 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 9.6 MB/s  0:00:00m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [matplotlib]6 [matplotlib]
Note: you may need to restart the kernel to use updated packages.


The next subsections provide different configuration scripts depending on your device.

You can uncomment and run the ones you need.

### Apple Silicon

Configuration for Apple Silicon devices.

In [ ]:
#pip install torch torchvision

  Using cached torchvision-0.25.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (5.4 kB)
  Using cached networkx-3.6.1-py3-none-any.whl.metadata (6.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.4/79.4 MB 12.4 MB/s  0:00:06m0:00:0100:01
Using cached torchvision-0.25.0-cp311-cp311-macosx_11_0_arm64.whl (1.9 MB)
Using cached networkx-3.6.1-py3-none-any.whl (2.1 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 12.0 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 12.5 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 12.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 12.7 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11/11 [torchvision] [torchvision]
Note: you may need to restart the kernel to use updated packages.


In [ ]:
"""
import torch

if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
    print("The MPS configuration worked. Computations will be performed on the Silicon chip when possible.")
else:
    device = torch.device("cpu")
    print("MPS device not found. Performance might be impacted.")
"""

The MPS configuration worked. Computations will be performed on the Silicon chip when possible.


### Linux, CUDA 12.6

Configuration for Linux devices equipped with CUDA 12.6.

In [ ]:
#pip install torch torchvision --index-url https://download.pytorch.org/whl/cu126

### Linux, ROCm 7.1

Configuration for Linux devices equipped with ROCm 7.1.

In [ ]:
#pip install torch torchvision --index-url https://download.pytorch.org/whl/rocm7.1

### Windows, CUDA 12.6

Configuration for Windows devices equipped with CUDA 12.6.

In [ ]:
#pip install torch torchvision --index-url https://download.pytorch.org/whl/cu126

### Windows, CPU

Configuration for Windows users choosing to run on CPU. Performance might be impacted.

In [ ]:
#pip install torch torchvision

### Other devices

If your situation doesn't fit any of the subsections, see the [PyTorch installation tutorial](https://pytorch.org/get-started/locally/).

## A 1D Helmholtz problem

## Implementation (1D)

The following chapter goes over the PyTorch implementation of a PINN solver for the Helmholtz problem.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

### Neural network

Let's initialize our neural network.

A typical PINN takes the physical coordinates of the problem as an input and outputs the solution.
In this example, the input is **x** (**1D** space variable) and the output is **u** (**1D** scalar output).

There are many architectural possibilities for the neural network. The simplest choice is a uniform fully-connected neural network with tanh activation functions. By default, each neuron has a weight and a bias.

We cannot use the tanh activation function after the last layer as we need a solution that can reach $1$ and $-1$.

TODO plot

In [ ]:
# Model declaration
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.stack = nn.Sequential(
            nn.Linear(1, 20),
            nn.Tanh(),
            nn.Linear(20, 20),
            nn.Tanh(),
            nn.Linear(20, 20),
            nn.Tanh(),
            nn.Linear(20, 1),
            #nn.Tanh(),
        )
    
    def forward(self, x):
        logits = self.stack(x)
        return logits

# Model initialization
model = Net()

Let's optimize our model parameters $\theta$ (weight & biases) iteratively using the Adam algorithm with a learning rate of $.001$ to minimize the mean-square-error loss function:

$$L_{\text{MSE}} \left(\theta, (x_i)_{i=1}^N, (y_i)_{i=1}^N \right) = \frac{1}{N} \sum_{i=1}^{N} \left( f_{\theta}(x_i) - y_i \right)^2 $$

Where $f_{\theta}$ is the model parametrized by the vector $\theta$, and we consider $N$ training samples:
- Training points $(x_i)_{i=1}^N$
- Respective solutions $(y_i)_{i=1}^N$

In [ ]:
# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Loss
loss = torch.nn.MSELoss() # the default reduction is "mean", dividing the loss by the number of samples

### PDE problem

Let's implement our PDE problem in Python.

In [ ]:
def residual_pde(X, U):
    du_dx = torch.autograd.grad(outputs=U,
                                inputs=X,
                                grad_outputs=torch.ones_like(U), # Shape information for batches
                                create_graph=True, # creating a graph for higher order derivatives
                                retain_graph=True,
                                )[0]
    du_dxx = torch.autograd.grad(outputs=du_dx,
                                 inputs=X,
                                 grad_outputs=torch.ones_like(du_dx), # Shape information for batches
                                 create_graph=True,
    )[0]
    return du_dxx + k**2 * U

## Further research...